## 7.1 작은 이미지 모아 놓은 데이터셋

### 7.1.1 CIFAR-10 다운로드
- 32 X 32 크기의 컬러(RGB) 이미지 6만개
- 1에서 10까지의 정수로 레이블
    * 0 : 비행기 / 1 : 자동차 / 2 : 새 / 3 : 고양이 / 4 : 사슴 / 5 : 강아지 / 6 : 개구리 / 7 : 말 / 8 : 배 / 9 : 트럭

In [1]:
from torchvision import datasets

In [2]:
data_path = r'C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/'
cifar10 = datasets.CIFAR10(data_path, train=True, download=True) # 훈련데이터용 데이터 객체 만듦
cifar10_val = datasets.CIFAR10(data_path, train=False, download=True)

100%|██████████████████████████████████████████████████████████████| 170498071/170498071 [00:14<00:00, 11704315.84it/s]


Extracting C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/cifar-10-python.tar.gz to C:\Users\knuyh\Desktop\민지\스터디\파이토치 딥러닝 마스터\Minji\Part01\CH07\Data/
Files already downloaded and verified


In [3]:
type(cifar10).__mro__

(torchvision.datasets.cifar.CIFAR10,
 torchvision.datasets.vision.VisionDataset,
 torch.utils.data.dataset.Dataset,
 typing.Generic,
 object)

### 7.1.2 데이터셋 클래스
- torch.utils.data.Dataset의 서브클래스
    * Dataset은 데이터를 직접 들고 있지는 않지만, __ len __, __getitem__을 통해 접근할 수 있다.
        - __ len__ : 데이터셋의 아이템 수 반환
        - __ getitem__ : 샘플과 레이블(정수 인덱스)로 이루어진 아이템 반환

In [4]:
len(cifar10)

50000

In [5]:
# dataset에 __getitem__메소드가 구현되어 있으므로 개별 아이템에 접근할 때 표준 서브스크립트에 해당하는 색인용 튜플과 리스트 사용 가능
# 자동차에 해당하는 정수값이 1인 PIL 형식의 이미지
img, label = cifar10[99]
img, label

(<PIL.Image.Image image mode=RGB size=32x32>, 1)

In [6]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']
class_names[label]

'automobile'

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
plt.imshow(img)
plt.show()

### 7.1.3 데이터 변환
- matplotlib으로 출력하기 위헤 permute() 사용하여 C X H X W를 H X W X C 형태로 변환

In [7]:
# PIL 이미지 -> 텐서로 변환
from torchvision import transforms
dir(transforms)

['AugMix',
 'AutoAugment',
 'AutoAugmentPolicy',
 'CenterCrop',
 'ColorJitter',
 'Compose',
 'ConvertImageDtype',
 'ElasticTransform',
 'FiveCrop',
 'GaussianBlur',
 'Grayscale',
 'InterpolationMode',
 'Lambda',
 'LinearTransformation',
 'Normalize',
 'PILToTensor',
 'Pad',
 'RandAugment',
 'RandomAdjustSharpness',
 'RandomAffine',
 'RandomApply',
 'RandomAutocontrast',
 'RandomChoice',
 'RandomCrop',
 'RandomEqualize',
 'RandomErasing',
 'RandomGrayscale',
 'RandomHorizontalFlip',
 'RandomInvert',
 'RandomOrder',
 'RandomPerspective',
 'RandomPosterize',
 'RandomResizedCrop',
 'RandomRotation',
 'RandomSolarize',
 'RandomVerticalFlip',
 'Resize',
 'TenCrop',
 'ToPILImage',
 'ToTensor',
 'TrivialAugmentWide',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_functional_pil',
 '_functional_tensor',
 '_presets',
 'autoaugment',
 'functional',
 'transforms']

In [8]:
to_tensor = transforms.ToTensor()
img_t = to_tensor(img)
img_t.shape # C X H X W

torch.Size([3, 32, 32])

In [9]:
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform = transforms.ToTensor())

In [10]:
img_t, _ = tensor_cifar10[99]
type(img_t) # PIL 타입이 아닌 tensor로 리턴

torch.Tensor

In [11]:
img_t.shape, img_t.dtype

(torch.Size([3, 32, 32]), torch.float32)

In [12]:
img_t.min(), img_t.max() # PIL이미지는 0 ~ 255(채널당 8비트) 범위, ToTensor 변환으로 데이터가 채널당 32비트 부동소수점 형태가 되면서 0.0에서 1.0 사이로 범위 줄어듦

(tensor(0.), tensor(1.))

### 7.1.4 데이터 정규화
- -1 ~ +1 혹은 -2 ~ +2 사이에서 선형인 활성 함수 택하고 데이터를 같은 범위에서 평균을 가지게 한다면 뉴런은 0이 아닌 기울기를 가지게 되므로 빨리 학습할 수 있다.
- 각 채널을 정규화해 동일한 분산을 가지게 된다면, 채널 정보가 동일한 학습률로 경사 하강을 통해 섞인다.
- transforms.Normalize ; mean, stdev 계산하지 않음

In [13]:
import torch
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
imgs.shape

torch.Size([3, 32, 32, 50000])

In [14]:
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download=False, transform = transforms.ToTensor())